In [ ]:
from os import path
import vtk
from vtk.util.numpy_support import vtk_to_numpy
import matplotlib.pyplot as plt
import numpy as np

import sklearn.neighbors as neighbors

In [ ]:
def extract_line(filename):
    # Read the VTP file
    reader = vtk.vtkXMLPolyDataReader()
    reader.SetFileName(filename)
    reader.Update()

    # Extract the polygon data
    polydata = reader.GetOutput()

    # Apply a filter to connect contiguous line segments
    # (This step is necessary since otherwise we would have many small line elements)
    strip = vtk.vtkStripper()
    strip.SetInputData(polydata)
    strip.SetJoinContiguousSegments(True)
    strip.Update()

    # Retrieve the filter output
    filtered = strip.GetOutput()

    # Extract Points
    point_coordinates = vtk_to_numpy(filtered.GetPoints().GetData())

    # Extract Line data
    lines = filtered.GetLines()
    lines_array = vtk_to_numpy(lines.GetData())

    # Connectivity array and offset array are only important if the line is not contiguous
    #connectivity_array = vtk_to_numpy(lines.GetConnectivityArray())
    #offset_array = vtk_to_numpy(lines.GetOffsetsArray())

    # Extract the surface line (as separate x, y and z array)
    return [np.array(d) for d in point_coordinates[lines_array[1:]].T]

# Extract interface lines
tx, ty, _ = extract_line(path.join("output_8bit", "Interface_0_0.vtp"))
x, y, _ = extract_line(path.join("output_8bit", "Interface_1_0.vtp"))

# Plot the line
fig, ax = plt.subplots(figsize=(9,16))
ax.plot(tx, ty, "x", label="Trench")
ax.plot(x, y, label="Oxide")

# Label plot
ax.set_xlabel("x [nm]")
ax.set_ylabel("y [nm]")
ax.legend(loc="upper center")
plt.axis("scaled")
plt.show()

In [ ]:
nbrs = neighbors.NearestNeighbors(n_neighbors=1, metric="euclidean").fit(np.vstack([x, y]).T)
distances, indices = nbrs.kneighbors(np.vstack([tx, ty]).T)

In [ ]:
# Plot the line
fig, ax = plt.subplots(figsize=(9,16))
ax.plot(tx, ty, label="Trench")
ax.plot(x, y, label="Oxide")

for mx, my, r in zip(tx, ty, distances):
    ax.add_patch(plt.Circle((mx, my), r, color='b', fill=False, clip_on=True))
# Label plot
ax.set_xlabel("x [nm]")
ax.set_ylabel("y [nm]")
ax.legend(loc="upper center")
plt.axis("scaled")
plt.show()

In [ ]:
sidewall_idx = np.where(np.bitwise_and(np.bitwise_and(tx < 0, ty < -0.5), ty > np.min(ty)+0.5))
sidewall_x, sidewall_y = tx[sidewall_idx], ty[sidewall_idx]

In [ ]:
plt.plot(-ty[sidewall_idx], distances[sidewall_idx])
plt.xlabel("Trench depth [nm]")
plt.ylabel("Deposition thickness [nm]")
#plt.axis("scaled")
plt.show()